## Data cleasning and preprocessing of the option pricing data

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import yfinance as yf
import sqlite3


In [2]:
# Read in the data from Jan 2023 to June 2023
df_2023_h1 = pd.DataFrame()
for i in [202301, 202302, 202303, 202304,  202305]:
    df_2023_h1 = pd.concat([df_2023_h1, pd.read_table(f'data/spy_eod_{i}.txt', sep=',')], ignore_index=True)
df_2023_h1.columns = df_2023_h1.columns.str.strip()

/var/folders/lc/cr1vnr593_l4t1814jghhy980000gn/T/ipykernel_800/79976602.py:4: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023_h1 = pd.concat([df_2023_h1, pd.read_table(f'data/spy_eod_{i}.txt', sep=',')], ignore_index=True)


In [3]:
# also drop expiration date later than 2024
df_2023_h1 = df_2023_h1[df_2023_h1['[EXPIRE_DATE]'] <= ' 2023-12-31']
df_2023_h1 = df_2023_h1.reset_index()

In [5]:
# get adj close data from yfinance
target = pd.DataFrame(yf.download(['SPY'], start="2023-01-01", end="2023-12-31")['Adj Close'])
target

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-01-03,375.118713
2023-01-04,378.014709
2023-01-05,373.700256
2023-01-06,382.269989
2023-01-09,382.053284
...,...
2023-12-22,473.649994
2023-12-26,475.649994
2023-12-27,476.510010


In [6]:
df_2023_h1['[EXPIRE_DATE]'] = df_2023_h1['[EXPIRE_DATE]'].str.strip().astype('datetime64[ns]')
df_2023_h1['[EXPIRE_DATE]'].unique()

<DatetimeArray>
['2023-01-03 00:00:00', '2023-01-04 00:00:00', '2023-01-05 00:00:00',
 '2023-01-06 00:00:00', '2023-01-09 00:00:00', '2023-01-10 00:00:00',
 '2023-01-11 00:00:00', '2023-01-12 00:00:00', '2023-01-13 00:00:00',
 '2023-01-20 00:00:00',
 ...
 '2023-06-01 00:00:00', '2023-06-05 00:00:00', '2023-06-06 00:00:00',
 '2023-06-07 00:00:00', '2023-06-08 00:00:00', '2023-07-07 00:00:00',
 '2023-06-12 00:00:00', '2023-06-13 00:00:00', '2023-06-14 00:00:00',
 '2023-11-17 00:00:00']
Length: 125, dtype: datetime64[ns]

In [7]:
target['[EXPIRE_DATE]'] = target.index
target['[EXPIRE_DATE]'].astype('datetime64[ns]')

Date
2023-01-03   2023-01-03
2023-01-04   2023-01-04
2023-01-05   2023-01-05
2023-01-06   2023-01-06
2023-01-09   2023-01-09
                ...    
2023-12-22   2023-12-22
2023-12-26   2023-12-26
2023-12-27   2023-12-27
2023-12-28   2023-12-28
2023-12-29   2023-12-29
Name: [EXPIRE_DATE], Length: 250, dtype: datetime64[ns]

In [8]:
df_2023_h1 = pd.merge(df_2023_h1, target, on = '[EXPIRE_DATE]')

In [9]:
df_2023_h1.columns

Index(['index', '[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
       '[QUOTE_TIME_HOURS]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]',
       '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]',
       '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_SIZE]',
       '[P_LAST]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[STRIKE_DISTANCE]',
       '[STRIKE_DISTANCE_PCT]', 'Adj Close'],
      dtype='object')

In [10]:
df_2023_h1['maturity_time'] = df_2023_h1['[EXPIRE_DATE]'] - df_2023_h1['[QUOTE_DATE]'].astype('datetime64[ns]')
# Convert the Timedelta to total seconds and then divide by the number of seconds in a day to get days as a float
df_2023_h1['maturity_time'] = df_2023_h1['maturity_time'].dt.total_seconds() / (60 * 60 * 24 * 365)


In [11]:
import math
df_2023_h1['discounted_price'] = df_2023_h1['Adj Close']
for i in range(len(df_2023_h1)):
    df_2023_h1['discounted_price'] = (df_2023_h1['[STRIKE]']-df_2023_h1['Adj Close'])*math.exp(df_2023_h1['maturity_time'][i]*(-0.04))

In [12]:
df_2023_h1

,index,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],...,[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT],Adj Close,maturity_time,discounted_price
0,0,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.96551,...,0.00072,-0.00483,-0.00015,1.210050,0.000000,70.8,0.186,375.118713,0.000000,-63.916774
1,1,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.96015,...,0.00104,-0.00487,0.00000,0.996160,0.000000,60.8,0.160,375.118713,0.000000,-54.101350
2,2,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.95788,...,0.00105,-0.00538,-0.00007,0.911990,,56.8,0.149,375.118713,0.000000,-50.175181
3,3,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.96337,...,0.00115,-0.00500,-0.00048,0.890650,0.000000,55.8,0.147,375.118713,0.000000,-49.193639
4,4,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.95600,...,0.00043,-0.00533,-0.00044,0.870040,0.000000,54.8,0.144,375.118713,0.000000,-48.212096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331604,402970,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-11-17,1700254800,170.04,0.46263,...,1.09166,-0.03382,-0.83587,0.151640,,12.2,0.029,448.969696,0.465753,-18.619560
331605,402971,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-11-17,1700254800,170.04,0.41652,...,1.04196,-0.02957,-0.84217,0.146010,,17.2,0.041,448.969696,0.465753,-13.711848
331606,402972,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-11-17,1700254800,170.04,0.36824,...,0.95627,-0.02495,-0.81425,0.139300,,22.2,0.053,448.969696,0.465753,-8.804136
331607,402973,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-11-17,1700254800,170.04,0.31986,...,0.84518,-0.02035,-0.76245,0.137080,,27.2,0.065,448.969696,0.465753,-3.896425


In [20]:
type(df_2023_h1['[QUOTE_DATE]'][0])

str

In [31]:
df_2023_h1.columns

Index(['index', '[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
       '[QUOTE_TIME_HOURS]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]',
       '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]',
       '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_SIZE]',
       '[P_LAST]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[STRIKE_DISTANCE]',
       '[STRIKE_DISTANCE_PCT]', 'Adj Close', 'maturity_time', 'discount_rate',
       'discounted_price'],
      dtype='object')

In [21]:
df_2023_h1 = df_2023_h1[['[QUOTE_UNIXTIME]', '[EXPIRE_UNIX]', '[STRIKE]', '[UNDERLYING_LAST]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]','[C_BID]', '[C_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[P_BID]', '[P_ASK]', 'discounted_price']]
for column in df_2023_h1.columns:
    print(type(df_2023_h1[column][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [22]:
df_2023_h1 = df_2023_h1.replace(r'^\s*$', 0, regex=True)

In [23]:
for column in ['[C_IV]', '[C_VOLUME]', '[P_IV]', '[P_VOLUME]']:
    df_2023_h1[column] = df_2023_h1[column].str.strip().astype('float64')
    print(type(df_2023_h1[column][0]))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [24]:
df_2023_h1 = df_2023_h1.fillna(0)

In [25]:
# Basic normalization and standardization
# run block of code and catch warnings
import warnings
from sklearn.preprocessing import StandardScaler
with warnings.catch_warnings():
	# ignore all caught warnings
	warnings.filterwarnings("ignore")
	# execute code that will generate warnings
	# dont standardize unixtime '[QUOTE_UNIXTIME]', '[EXPIRE_UNIX]', if it does what I think it does. 
	numeric_cols = ['[STRIKE]', '[UNDERLYING_LAST]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]','[C_BID]', '[C_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[P_BID]', '[P_ASK]']  # not sure about all this, we ball
	scaler = StandardScaler()
	df_2023_h1[numeric_cols] = scaler.fit_transform(df_2023_h1[numeric_cols])

In [34]:
 # split target into two df
target = df_2023_h1['discounted_price']
df_2023_h1 = df_2023_h1.drop('discounted_price', axis=1)

In [35]:
# output the df_2023_h1 to a csv file, output to the desktop
df_2023_h1.to_csv(r'data/df_2023_h1.csv', index = False, header=True)
target.to_csv(r'data/target.csv', index = False, header=True)

In [36]:
# output to sqlite database if anyone cares
conn = sqlite3.connect("data/tables.db")
df_2023_h1.to_sql("df_2023_h1_train", conn, if_exists = "replace", index=False)
target.to_sql("df_2023_h1_target", conn, if_exists = "replace", index=False)
conn.close()